<img src="https://cdn-images-1.medium.com/max/800/1*6UcQdP-1yFoUotb-QnDWsg.gif">

<img src="https://sites.google.com/site/learnphotographybasics/_/rsrc/1239834796658/Home/color-bit-depth-channels-info-pallet/3%20channels%20over%20lay.png">

<img src="https://thumbs.dreamstime.com/z/rgb-bijkomend-kleurenmodel-53432489.jpg">

<img src="https://s3-ap-south-1.amazonaws.com/av-blog-media/wp-content/uploads/2017/06/28011851/conv.gif">

In [1]:
from skimage.exposure import rescale_intensity
import numpy as np
import argparse
import cv2
import math
from matplotlib import pyplot as plt

In [2]:
def convolve(image, kernel,ker_mul,iterations):
    
    if iterations==0:
        return image
    else:
        (iH, iW) = image.shape[:2]
        (kH, kW) = kernel.shape[:2]

        pad = (kW - 1) / 2
        image = cv2.copyMakeBorder(image, pad, pad, pad, pad,cv2.BORDER_REPLICATE)
        output = np.zeros((iH, iW), dtype="float32")

        for y in range(iH):
            for x in range(iW):
                roi = image[y :y + kH, x:x + kW]
                k = (roi * kernel).sum()
                k=k/ker_mul
                output[y,x] = k

        output = rescale_intensity(output, in_range=(0, 255))
        output = (output * 255).astype("uint8")
        return convolve(output,kernel,ker_mul,iterations-1)

In [75]:
def find_shape(image, kernel):
    (iH, iW) = image.shape[:2]
    (kH, kW) = kernel.shape[:2]

    pad = (kW - 1) / 2
    image = cv2.copyMakeBorder(image, pad, pad, pad, pad,cv2.BORDER_REPLICATE)
    output = np.zeros((iH, iW), dtype="float32")
    prev_sum=0
    for y in range(iH):
        for x in range(iW):
            roi = image[y :y + kH, x:x + kW]
            k = (roi * kernel).sum()
            if k> prev_sum:
                prev_sum=k
                y1,yw,x1,xw=y,y + kH,x,x + kW
    
    return (cv2.rectangle(image.copy(), (x1, y1), (xw, yw), (255,0,0), 2))
    

In [3]:
%matplotlib
def plot(*images):
    ncols=2
    nrows=int(math.ceil(len(images)/2.0))
    fig, ax = plt.subplots(nrows, ncols,gridspec_kw = {'wspace':0, 'hspace':0},facecolor='.5')
    plt.axis('off')
    for i in range(0,len(images)):
        plt.subplot(nrows,ncols,i+1),plt.imshow(images[i],cmap='Greys')
        plt.axis('off')
    plt.show()

Using matplotlib backend: TkAgg


In [28]:
def read_image(path,invert=0):
    image= cv2.imread(path,0)
    if invert==1:
        ret,image = cv2.threshold(image,127,255,cv2.THRESH_BINARY_INV)
    plot(image)
    return image

In [72]:
sobelX = np.array((
    [-1, 0, 1],
    [-2, 0, 2],
    [-1, 0, 1]), dtype="int")
sharpen = np.array((
    [0, -1, 0],
    [-1, 5, -1],
    [0, -1, 0]), dtype="int")
sobelY = np.array((
    [-1, -2, -1],
    [0, 0, 0],
    [1, 2, 1]), dtype="int")
extendY=np.array((
    [0, 1, 0],
    [0, 1, 0],
    [0, 1, 0]), dtype="int")
extendX=np.transpose(extendY)
axisel=np.array((
    [1, 0, 0],
    [0, 1, 0],
    [0, 0, 1]), dtype="int")
erode=np.array((
    [1, 1, 1],
    [1, 1, 1],
    [1, 1, 1]), dtype="int")
mouse_k=np.array((
    [0,0,0,1,1,1,0],
    [0,0,1,1,0,0,0],
    [0,1,1,0,0,0,0],
    [0,1,1,0,0,0,0],
    [0,1,1,0,0,0,0],
    [0,1,1,0,0,0,0],
    [0,0,1,0,0,0,0]), dtype="int")

In [83]:
plus=read_image("./Plus_sign.jpg",1)

In [95]:
out1=convolve(plus,sobelY,1,5)

In [96]:
plot(plus,out1)

In [ ]:
cv2.imshow('image',out)
cv2.waitKey(0)
cv2.destroyAllWindows()

In [63]:
plot(plus,out)

In [92]:
mouse=read_image("./mouse.jpg",1)

In [64]:
out=convolve(mouse,mouse_k,1,1)

In [67]:
plot(mouse,out)

In [94]:
shape=find_shape(mouse,mouse_k)
plot(mouse)